<a href="https://colab.research.google.com/github/toluntosun21/ML-notebooks/blob/master/LetterClassification-SVM/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SVM**

In this example, I will separate handwritten 'B' characters from 'P' characters in UCI letter recognition dataset,using Support Vector Machines (SVM).

In [0]:
#upload the data first
from google.colab import files
uploaded = files.upload()

MessageError: ignored

In [0]:
#import pandas to read the input as csv
import pandas as pd

Bs = pd.read_csv('Bs.csv', header=None)
Ps = pd.read_csv('Ps.csv', header=None)
#add the classes
Bs.insert(16,'y',0) #0 denotes B
Ps.insert(16,'y',1) #1 denotes P
print(Bs)
print(Ps)
chars=pd.concat([Bs,Ps])
print(chars)
numRows=len(chars)
numCols=len(chars.columns)
numFeatures=(numCols-1)
print(numRows)
print(numCols)

     0   1  2  3   4   5  6  7  8   9  10  11  12  13  14  15  y
0    4   2  5  4   4   8  7  6  6   7   6   6   2   8   7  10  0
1    5   9  7  7  10   9  8  4  4   6   8   6   6  11   8   7  0
2    3   3  3  4   3   7  7  5  5   7   6   6   5   8   5  10  0
3    5   9  7  7   7   8  8  3  6  10   5   6   3   7   6   8  0
4    4   2  5  4   4   7  7  5  6   7   6   6   2   8   7  10  0
..  ..  .. .. ..  ..  .. .. .. ..  ..  ..  ..  ..  ..  ..  .. ..
761  6  10  8  8   7   7  7  6  6   9   7   6   3   8   8   9  0
762  2   5  4  3   3   8  8  3  5  10   5   6   3   7   5   9  0
763  6   9  8  6   7  10  6  3  6  10   3   7   4   7   5  12  0
764  4   4  5  7   4   6  6  9  7   6   6   7   2   8   9  10  0
765  4   8  6  6   5   7  8  6  6  10   6   6   3   8   7   8  0

[766 rows x 17 columns]
     0   1   2  3  4  5   6  7  8   9  10  11  12  13  14  15  y
0    3   6   4  4  2  4  14  8  1  11   6   3   0  10   4   8  1
1    8  14   7  8  4  5  10  6  3  12   5   4   4  10   4   8  1


In [0]:
#random split for test, valid, train sets
def shuffle(input,frac):
  shuffle=input.sample(frac=1).reset_index(drop=True)
  datalen=len(input)
  firstSplitEnd=int(datalen*frac)
  firstSet=shuffle[0:trainend]
  secondSet=shuffle[firstSplitEnd:datalen]
  return firstSet, secondSet

trainValid, test = shuffle(chars, 0.8) #0.2 of data is reserved for test set
print(trainValid)
print(test)

     0   1  2  3  4  5   6  7  8   9  10  11  12  13  14  15  y
0    7   9  6  4  5  9   7  3  5   9   4   7   7   5   7   8  0
1    4  11  6  8  8  7   7  6  5   7   6   6   3   8   6  10  0
2    2   4  4  3  2  6  10  3  4  12   5   3   1  10   2   8  1
3    4   5  5  4  4  7   7  5  6   7   6   6   2   8   6  10  0
4    2   3  2  2  2  7   8  5  5   7   5   6   2   8   5   9  0
..  ..  .. .. .. .. ..  .. .. ..  ..  ..  ..  ..  ..  ..  .. ..
936  4   9  4  6  4  3  14  6  1  12   7   3   0   9   3   8  1
937  3   2  4  4  3  7   7  5  5   6   6   5   2   8   6  10  0
938  2   7  3  5  2  6   6  9  7   6   7   7   2   8   8  10  0
939  6  11  9  8  5  6  12  5  4  12   5   2   1  10   3   9  1
940  3   3  3  5  3  6   6  9  6   6   6   7   2   8   9  10  0

[941 rows x 17 columns]
      0   1  2  3  4   5   6   7  8   9  10  11  12  13  14  15  y
1255  2   4  4  3  3   8   7   3  6  10   5   6   2   8   4  10  0
1256  6   8  8  6  5   5  14   6  2  12   6   2   1  10   3   7  1
1257  

First, let's try linear kernel

In [0]:
#learning with sckitlearn svm classifier with linear kernel
from sklearn import svm

def predictAndTest(classifier,testData):
  predictions=classifier.predict(testData.iloc[:,:numFeatures].values.tolist()) #predict the validation set classes
  success=0
  for i in range(0,len(predictions)):
    if(predictions[i]==testData.y.iloc[i]):
      success+=1
  return success/len(testData)

def validationScoreLinear(C_, trainCount):#with given C, make trainCount experiments and report the result
  clf = svm.SVC(kernel='linear', C = C_)
  cummSuccess=0
  for k in range(trainCount):
    trainset,validset = shuffle(trainValid,0.75) #get a random train set
    clf.fit(trainset.iloc[:,:numFeatures].values.tolist(), trainset.y) #train the test set
    cummSuccess+=predictAndTest(clf,validset)
  return cummSuccess/trainCount

C=0.00001
maxScore=0
maxc=C
for i in range(0,9):
  score=validationScoreLinear(C, 100)
  if(score>maxScore):
    maxScore=score
    maxc=C
  print(C, ' : ', score)
  C = C*10


print('max is: ', maxc, ' with ', maxScore)  

1e-05  :  0.5075847457627118
0.0001  :  0.946228813559322
0.001  :  0.96406779661017
0.01  :  0.9761440677966102
0.1  :  0.9879237288135595
1.0  :  0.9933898305084745
10.0  :  0.9938559322033893
100.0  :  0.9978813559322033
1000.0  :  1.0
max is:  1000.0  with  1.0


The above code prints out the best C value as 100(it may change in your run so I am doing it parametrically). Let's train the set again with C=100 and report the accuracy over test set

In [0]:
clf = svm.SVC(kernel='linear', C = maxc)
clf.fit(trainValid.iloc[:,:numFeatures].values.tolist(), trainValid.y) #train the test set
score=predictAndTest(clf,testset)

print('Accuracy: ', score)


Accuracy:  0.9872611464968153


The accuracy is around 0.987 !!!
It may differ depending on the split

Next, let's try radial basis function kernel (RBF), this time I will use gridsearch class of skilearn for tuning parameters C and $\gamma$

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

trainset=trainValid

grid_values = {'gamma': [2**(-9),2**(-8),2**(-7),2**(-6),2**(-5),2**(-4),2**(-3),2**(-2),2**(-1),2],'C':[0.0001,0.001, 0.01,0.1,1,10,100,1000]}
clf = svm.SVC(kernel='rbf')
grid = GridSearchCV(clf, param_grid = grid_values, scoring = 'accuracy', cv=10, return_train_score=False)
grid.fit(trainset.iloc[:,:numFeatures].values.tolist(), trainset.y) #train the test set
#print(grid.cv_results_) #comment out to see cross validation results
print('best parameters: ', grid.best_params_, ' with score ', grid.best_score_)





best parameters:  {'C': 100, 'gamma': 0.015625}  with score  0.997874601487779


the best parameters found are C=100 and $\gamma$=0.015625

again, it may differ depending on the split


---



In [0]:
#let's train the data with best parameters and report the test result on test split
clf = svm.SVC(kernel='rbf', gamma=grid.best_params_['gamma'], C=grid.best_params_['C'])
clf.fit(trainset.iloc[:,:numFeatures].values.tolist(), trainset.y) #train the test set
score=predictAndTest(clf,testset)

print('Accuracy: ', score)
#we could also use the library function, they will report the same number
#print(grid.score(testset.iloc[:,:numFeatures].values.tolist(), testset.y))



Accuracy:  0.9904458598726115


Accuracy is over 0.99 !!